In [2]:
!which pip

/home/gellert/Documents/UNI/DeepLearning/hackaton/.venv/bin/pip


In [2]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

Cloning into 'hackaton'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 81 (delta 7), reused 0 (delta 0), pack-reused 62 (from 2)
Receiving objects: 100% (81/81), 105.83 MiB | 2.57 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [3]:
%cd hackaton/

[Errno 2] No such file or directory: 'hackaton/'
/home/gellert/Documents/UNI/DeepLearning/hackaton


/home/gellert/Documents/UNI/DeepLearning/hackaton/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [4]:
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


/bin/bash: line 1: gdown: command not found


In [4]:
!ls -lh datasets

total 16K
drwxrwxr-x 2 gellert gellert 4.0K May 23 15:37 A
drwxrwxr-x 2 gellert gellert 4.0K May 23 15:40 B
drwxrwxr-x 2 gellert gellert 4.0K May 24 13:58 C
drwxrwxr-x 2 gellert gellert 4.0K May 23 15:26 D


In [71]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository

import sys
import os

# Add the parent directory of src to the system path
sys.path.append(os.path.abspath('./'))

from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse
from torch.optim.lr_scheduler import CosineAnnealingLR

# Set the random seed
set_seed()


In [72]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [73]:
def train(data_loader, model, optimizer, device, save_checkpoints, checkpoint_path, current_epoch):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for data in tqdm(data_loader, desc="Iterating training graphs", unit="batch"):
        data = data.to(device)
        optimizer.zero_grad()
        node_emb, output = model(data)
        loss = model.loss(node_emb, output, data)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += (pred == data.y).sum().item()
        total += data.y.size(0)

    # Save checkpoints if required
    if save_checkpoints:
        checkpoint_file = f"{checkpoint_path}_epoch_{current_epoch + 1}.pth"
        torch.save(model.state_dict(), checkpoint_file)
        print(f"Checkpoint saved at {checkpoint_file}")

    return total_loss / len(data_loader),  correct / total

In [74]:
from sklearn.metrics import f1_score

def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    true_labels = []
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            node_emb, output = model(data)
            pred = output.argmax(dim=1)
            predictions.extend(pred.cpu().numpy())
            
            if calculate_accuracy:
                true_labels.extend(data.y.cpu().numpy())
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
    if calculate_accuracy:
        f1 = f1_score(true_labels, predictions, average='weighted')  
        accuracy = correct / total
        return  total_loss / len(data_loader), accuracy, f1
    return predictions

In [75]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [76]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot losscolabes, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [77]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [93]:
def get_arguments():
    args = {}
    # args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['train_path'] = "./datasets/A/train.json.gz"

    # args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['test_path'] = "./datasets/A/test.json.gz"
    
    # args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['num_checkpoints'] = 5
    
    # args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['device'] = 0
    
    # args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin')
    args['gnn'] = "gin-virtual"
    
    # args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['drop_ratio'] = 0.3
    
    # args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['num_layer'] = 5
    
    # args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['emb_dim'] = 64
    
    # args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['batch_size'] = 32
    
    # args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['epochs'] = 2
    args["warmup_epochs"] = 1
    
    # args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (Noisy CE)", default=1, type_cast=int)
    args['baseline_mode'] = 2
    
    # args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)
    args['noise_prob'] = 0.2

    args["num_voters"] = 2


    
    return argparse.Namespace(**args)


In [94]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Arguments received:
train_path: ./datasets/A/train.json.gz
test_path: ./datasets/A/test.json.gz
num_checkpoints: 5
device: 0
gnn: gin-virtual
drop_ratio: 0.3
num_layer: 5
emb_dim: 64
batch_size: 32
epochs: 2
warmup_epochs: 1
baseline_mode: 2
noise_prob: 0.2
num_voters: 2


In [95]:
class NoisyCrossEntropyLoss(torch.nn.Module):
    def __init__(self, p_noisy):
        super().__init__()
        self.p = p_noisy
        self.ce = torch.nn.CrossEntropyLoss(reduction='none')

    def forward(self, logits, targets):
        losses = self.ce(logits, targets)
        weights = (1 - self.p) + self.p * (1 - torch.nn.functional.one_hot(targets, num_classes=logits.size(1)).float().sum(dim=1))
        return (losses * weights).mean()

In [112]:
from src.conv import GNN_node, GNN_node_Virtualnode
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool, GlobalAttention, Set2Set
import torch.nn.functional as F
from torch import nn


class GNN(torch.nn.Module):

    def __init__(self, num_class, num_layer = 5, emb_dim = 300, latent_dim = 16,
                    gnn_type = 'gin', virtual_node = True, residual = False, drop_ratio = 0.5, JK = "last", graph_pooling = "mean"):
        '''
            num_tasks (int): number of labels to be predicted
            virtual_node (bool): whether to add virtual node or not
        '''

        super(GNN, self).__init__()

        self.num_layer = num_layer
        self.drop_ratio = drop_ratio
        self.JK = JK
        self.emb_dim = emb_dim
        self.num_class = num_class
        self.graph_pooling = graph_pooling

        if self.num_layer < 2:
            raise ValueError("Number of GNN layers must be greater than 1.")

        ### GNN to generate node embeddings
        if virtual_node:
            self.gnn_node = GNN_node_Virtualnode(num_layer, emb_dim, JK = JK, drop_ratio = drop_ratio, residual = residual, gnn_type = gnn_type)
        else:
            self.gnn_node = GNN_node(num_layer, emb_dim, JK = JK, drop_ratio = drop_ratio, residual = residual, gnn_type = gnn_type)


        ### Pooling function to generate whole-graph embeddings
        if self.graph_pooling == "sum":
            self.pool = global_add_pool
        elif self.graph_pooling == "mean":
            self.pool = global_mean_pool
        elif self.graph_pooling == "max":
            self.pool = global_max_pool
        elif self.graph_pooling == "set2set":
            self.pool = Set2Set(emb_dim, processing_steps = 2)
        else:
            raise ValueError("Invalid graph pooling type.")
        
        self.node_emb = torch.nn.Linear(emb_dim, latent_dim)

        self.edge_attr_decoder = torch.nn.Sequential(
            torch.nn.Linear(latent_dim * 2, latent_dim),
            torch.nn.LeakyReLU(0.1),
            torch.nn.Linear(latent_dim, 7)
        )

        self.classifier = torch.nn.Linear(emb_dim, 6)

        self.criterion = NoisyCrossEntropyLoss(args.noise_prob)

    def decode(self, z, edge_index):
        adj_pred = torch.sigmoid(torch.mm(z, z.t()))
        
        edge_feat_input = torch.cat([z[edge_index[0]], z[edge_index[1]]], dim=1)
        edge_attr_pred = torch.sigmoid(self.edge_attr_decoder(edge_feat_input))
        
        return adj_pred, edge_attr_pred

    def forward(self, batched_data):
        h_node = torch.nn.functional.leaky_relu(self.gnn_node(batched_data), 0.1)
        h_graph = self.pool(h_node, batched_data.batch)

        node_emb = self.node_emb(h_node)
        return node_emb, self.classifier(h_graph)
    
    def recon_loss(self, z, data, alpha=0.1, beta=1, gamma=0.1):
        adj_pred, edge_attr_pred = self.decode(z, data.edge_index)
        adj_true = torch.zeros_like(adj_pred)
        adj_true[data.edge_index[0], data.edge_index[1]]

        adj_loss = F.binary_cross_entropy(adj_pred, adj_true)
        edge_attr_loss = F.mse_loss(edge_attr_pred, data.edge_attr)

        # kl_loss = -0.5 * torch.mean(torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1))
        return alpha * adj_loss + beta * edge_attr_loss

    def loss(self, node_emb, class_logits, data):
        classification_loss = self.criterion(class_logits, data.y)
        recon_loss = self.recon_loss(node_emb, data)

        loss = 0.1 * recon_loss + classification_loss
        return loss
    
class EnsembleModel(nn.Module):
    def __init__(self, models: list, weights, device):
        super().__init__()
        self.models = nn.ModuleList(models)
        # Trainable weights initialized equally
        self.register_buffer("weights", torch.tensor(weights, dtype=torch.float32))
        self.weights = (self.weights / self.weights.sum(dim=0)).to(device)

    def forward(self, x):
        outputs = [model(x)[1] for model in self.models]
        outputs = torch.stack(outputs)
        weighted_avg = (outputs * self.weights.view(-1, 1, 1)).sum(dim=0)
        return None, weighted_avg
    

In [82]:
full_dataset = GraphDataset(args.train_path, transform=add_zeros)

In [ ]:
def train_once(model, args, voter, full_dataset, test_dir_name, logs_folder, script_dir, device):
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    generator = torch.Generator().manual_seed(voter+1)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_voter_{voter}_best.pth")
    checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
    os.makedirs(checkpoints_folder, exist_ok=True)

    num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
        

    initial_lr = 1e-5
    target_lr = 1e-3
    minimum_lr = 1e-6

    optimizer = torch.optim.Adam(model.parameters(), lr=initial_lr)
    scheduler = CosineAnnealingLR(optimizer, T_max=args.epochs-args.warmup_epochs, eta_min=minimum_lr)

    num_epochs = args.epochs
    best_f1 = 0.0   

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    def get_lr(epoch):
        if epoch < args.warmup_epochs:
            lr = initial_lr + (target_lr - initial_lr) * (epoch / args.warmup_epochs)
        else:
            scheduler.step()
            lr = scheduler.get_last_lr()[-1]
        return lr


    for epoch in range(num_epochs):
        lr = get_lr(epoch)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        print(f"Current learning rate: {lr}")

        train_loss, train_acc = train(
            train_loader, model, optimizer, device,
            save_checkpoints=(epoch + 1 in checkpoint_intervals),
            checkpoint_path=os.path.join(checkpoints_folder, f"model_{test_dir_name}_voter_{voter}"),
            current_epoch=epoch
        )

        val_loss,val_acc, f1 = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Voter {voter}, Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Val f1: {f1:.4f}")
        logging.info(f"Voter {voter}, Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Val f1: {f1:.4f}")
        
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        
        if f1 > best_f1:
            best_f1 = f1
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")
    model.load_state_dict(torch.load(checkpoint_path))
    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, f"plots_voter_{voter}"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, f"plotsVal_voter_{voter}"))

    return model, best_f1


In [104]:
script_dir = os.getcwd() 
device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")

In [105]:
def get_model():
    if args.gnn == 'gin':
        model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
    elif args.gnn == 'gin-virtual':
        model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
    elif args.gnn == 'gcn':
        model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
    elif args.gnn == 'gcn-virtual':
        model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
    else:
        raise ValueError('Invalid GNN type')
    return model

In [ ]:


test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, f"training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)

if args.train_path:

    models, weights = [], []
    for voter in range(args.num_voters):
        model, weight = train_once(get_model(), args, voter, full_dataset, test_dir_name, logs_folder, script_dir, device)
        models.append(model)
        weights.append(weight)
    ensemble = EnsembleModel(models, weights, device)

    torch.save(ensemble.state_dict(), checkpoint_path)

        

Current learning rate: 1e-05


Iterating training graphs:   0%|          | 0/282 [00:00<?, ?batch/s]

Iterating training graphs: 100%|██████████| 282/282 [00:32<00:00,  8.56batch/s]


Checkpoint saved at /home/gellert/Documents/UNI/DeepLearning/hackaton/checkpoints/A/model_A_voter_0_epoch_1.pth


Iterating eval graphs: 100%|██████████| 71/71 [00:06<00:00, 10.40batch/s]
Voter 0, Epoch 1/2, Loss: 1.4850, Train Acc: 0.1730, Val Acc: 0.1760, Val f1: 0.0720
Voter 0, Epoch 1/2, Loss: 1.4850, Train Acc: 0.1730, Val Acc: 0.1760, Val f1: 0.0720
Voter 0, Epoch 1/2, Loss: 1.4850, Train Acc: 0.1730, Val Acc: 0.1760, Val f1: 0.0720
Voter 0, Epoch 1/2, Loss: 1.4850, Train Acc: 0.1730, Val Acc: 0.1760, Val f1: 0.0720
Voter 0, Epoch 1/2, Loss: 1.4850, Train Acc: 0.1730, Val Acc: 0.1760, Val f1: 0.0720
Voter 0, Epoch 1/2, Loss: 1.4850, Train Acc: 0.1730, Val Acc: 0.1760, Val f1: 0.0720
Voter 0, Epoch 1/2, Loss: 1.4850, Train Acc: 0.1730, Val Acc: 0.1760, Val f1: 0.0720
Voter 0, Epoch 1/2, Loss: 1.4850, Train Acc: 0.1730, Val Acc: 0.1760, Val f1: 0.0720
Voter 0, Epoch 1/2, Loss: 1.4850, Train Acc: 0.1730, Val Acc: 0.1760, Val f1: 0.0720


...

Iterating training graphs: 100%|██████████| 282/282 [00:33<00:00,  8.49batch/s]


Checkpoint saved at /home/gellert/Documents/UNI/DeepLearning/hackaton/checkpoints/A/model_A_voter_1_epoch_2.pth


Iterating eval graphs: 100%|██████████| 71/71 [00:06<00:00, 11.23batch/s]
Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489
Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489
Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489
Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489
Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489
Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489
Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489
Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489
Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489


Voter 1, Epoch 2/2, Loss: 1.4297, Train Acc: 0.2664, Val Acc: 0.2908, Val f1: 0.1489


TypeError: save() missing 1 required positional argument: 'f'

In [100]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [113]:
model = EnsembleModel([get_model() for _ in range(args.num_voters)], [1 for _ in range(args.num_voters)], device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

Iterating eval graphs: 100%|██████████| 74/74 [00:04<00:00, 15.13batch/s]

Predictions saved to /home/gellert/Documents/UNI/DeepLearning/hackaton/submission/testset_A.csv
